In [122]:
import networkx as nx
import numpy as np
from itertools import chain, combinations

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [67]:
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [68]:
len(list(powerset([1,2,3,4,5,6])))

64

In [69]:
def find_max_weight_edge(sim_graph):
    # Search for
    max_weight = -1
    tu = 0
    tv = 0
    vertex_count = sim_graph.number_of_nodes()
    for u in range(0,vertex_count):
        linked_nodes = sim_graph.adj[u]
        for v in list(linked_nodes):
            w = sim_graph.edges[u,v]["weight"]
            print("u="+str(u) + "," + str(v) + ",weight=" + str(w))
            if (w > max_weight):
                tu = u
                tv = v
                max_weight = w
    return tu,tv, max_weight

In [70]:
def delete_edges_from(graph, vertex):
    e = graph.adj[vertex]
    for linked in list(e):
        graph.remove_edge(vertex,linked)

In [71]:
def sort_by_metric_func(graph):
    return list(graph.nodes())

In [72]:
def get_metric_ranking(rankings, num):
    return rankings.index(num)

In [73]:
# Eqn 2 in diversification paper
def calc_compdist_from(graph, i, j, lam):
    dimension_dist = 0
    i_props = graph.nodes[i]
    j_props = graph.nodes[j]
    
    rankings = sort_by_metric_func(graph)
    ranking_i = get_metric_ranking(rankings, i)
    ranking_j = get_metric_ranking(rankings, j)
    print("ranking_i" + str(ranking_i))
    print("ranking_j" + str(ranking_j))
    
    # Get the properties
    for k,v in i_props.items():
        # Dimension values only
        if "d" in k:
            dist = (v - j_props[k])
            dimension_dist += (dist ** 2.0)
            
    comp_value = ranking_i + ranking_j + 2 * lam * np.sqrt(dimension_dist)
    print("dimension_dist = " + str(dimension_dist))
    print("comp_value = " + str(comp_value))
    return comp_value
    

In [74]:
# Need to add arbitrary vertex
def max_sum_divergence_approx(sim_graph, phy_count):
    Phy = nx.Graph() 
    lim = int(np.floor(phy_count/2))
    for i in range(0,lim):
        mu, mv, weight = find_max_weight_edge(sim_graph)
        if (mu != mv):
            print("Max weight: " + str(mu) + "->" + str(mv) + ",weight=" + str(weight))
            Phy.add_node(mu)
            Phy.add_node(mv)
            Phy.add_edge(mu,mv)
            Phy.edges[mu,mv]["weight"] = sim_graph.edges[mu,mv]["weight"]
            print("Number of edges in Phy=" + str(Phy.number_of_edges()))
            # remove other connected edges from mu and mv in the s\im_graph
            delete_edges_from(sim_graph, mu)
            delete_edges_from(sim_graph, mv)
            # check further
        
    # If phy_count is odd, add a random vertex
    # for now, only use an even phy_count
    return Phy

In [75]:
def gen_node_props(metrics_count, dim_count, metric_max, dim_max):
    nodeprops = {}
    metrics = []
    
    for di in range(0, dim_count):
        rand_dim = np.random.uniform(low=0, high=dim_max)
        nodeprops["d" + str(di)] = rand_dim
        
    # Generate the metrics
    for mi in range(0,metrics_count):
        rand_metric = np.random.uniform(low=0, high=metric_max)
        metrics.append(rand_metric)
        nodeprops["m" + str(mi)] = rand_metric
    return nodeprops

In [76]:
def create_random_sim_graph(node_count, metric_count, metric_max, dim_count, dim_max):
    Sim = nx.Graph();
    for i in range(0,node_count):
        Sim.add_nodes_from([(i, gen_node_props(metric_count, dim_count, metric_max, dim_max))])
    # add connections from all nodes, depending on their distance 
    for i in range(0, node_count):
        for j in range(0, node_count):
            if (i != j):
                Sim.add_edge(i,j)
                #weight = calc_compdist_from(Sim, i, j, lam)
                #Sim.edges[i,j]["weight"] = weight
                #print("Adding edge from " + str(i) + "->" + str(j) + ", weight = " + str(Sim.edges[i,j]["weight"]))
                #print(Sim.edges[i, j])
    print("Edges: = " + str(Sim.number_of_edges()))
    return Sim

In [77]:
max_dimension = 1 / np.sqrt(3)
sim = create_random_sim_graph(6, 2, 10.0, 3, max_dimension)
#phy = max_sum_divergence_approx(sim, 10)
#phy
sim

Edges: = 15


In [78]:
def ranking_func(graph,node_id):
    return node_id / len(graph.nodes())

In [111]:
def dist_func(graph, ni, nj):
    i_props = graph.nodes[ni]
    j_props = graph.nodes[nj]
    dimension_dist = 0.0
    for k,v in i_props.items():
    # Dimension values only
        if "d" in k:
            dist = (v - j_props[k])
            dimension_dist += (dist ** 2.0)
    return np.sqrt(dimension_dist)

In [113]:
def calc_subset_value_function(graph, subset_inds, lam):
    k = len(subset_inds)
    weight_sum = 0
    for ni in subset_inds:
    # ranking_func is w in the equation
        weight_sum += ranking_func(graph, ni)
    
    # Are we double counting the distance here?
    dist_sum = 0
    for ni in subset_inds:
        for nj in subset_inds:
            dist_sum += dist_func(graph, ni,nj)
    total = (k-1)*weight_sum + 2*lam*dist_sum  
    return total

In [91]:
all_subsets = list(powerset(sim.nodes()))

In [82]:
def subset_value(graph, subset, lam):
    value = calc_subset_value_function(graph, subset, lam)
    return (subset, value)

In [83]:
FIXED_SUBSET_COUNT = 5

In [84]:
def subset_cost(subset):
    return len(subset)

In [85]:
def meets_cost_constraint(subset):
    return (subset_cost(subset) < FIXED_SUBSET_COUNT)

In [115]:
def subsets_viable_sorted_by_cost(simgraph, lam):
    all_subsets = list(powerset(simgraph.nodes()))
    viable_subsets = filter(meets_cost_constraint, all_subsets)
    subset_info = map(lambda subset: subset_value(simgraph, subset, lam), viable_subsets)
    return sorted(subset_info, key=lambda info: info[1], reverse=True)

In [119]:
res = subsets_viable_sorted_by_cost(sim, 1.0)
list(res)

[((0, 2, 4, 5), 14.417205844520774),
 ((0, 2, 3, 4), 13.96153103309453),
 ((0, 2, 3, 5), 13.632665978150719),
 ((2, 3, 4, 5), 13.118946280121033),
 ((0, 1, 2, 4), 12.66426034443312),
 ((0, 3, 4, 5), 12.55136455520724),
 ((0, 1, 2, 5), 12.49607353390177),
 ((0, 1, 2, 3), 11.982061954506147),
 ((0, 1, 4, 5), 11.687244638719172),
 ((0, 1, 3, 4), 11.341239450167063),
 ((1, 2, 3, 4), 11.213732418448602),
 ((1, 2, 4, 5), 11.11990609591193),
 ((0, 1, 3, 5), 11.055504622327977),
 ((1, 2, 3, 5), 10.845208358995553),
 ((1, 3, 4, 5), 10.277922330310131),
 ((0, 2, 4), 7.694065345440879),
 ((0, 2, 5), 7.5807837308881485),
 ((0, 2, 3), 7.018385759012048),
 ((2, 3, 4), 6.881602846289788),
 ((2, 4, 5), 6.836162916233592),
 ((0, 4, 5), 6.723399696478932),
 ((2, 3, 5), 6.56798398281536),
 ((0, 1, 2), 6.3501970373042),
 ((0, 3, 4), 6.329008115446344),
 ((0, 3, 5), 6.09817848358588),
 ((3, 4, 5), 5.952142814903323),
 ((0, 1, 4), 5.933291921499379),
 ((0, 1, 5), 5.8631405340513725),
 ((0, 1, 3), 5.41041218

In [121]:
# Plot the surface and the graph between them

In [127]:
 np.array([props[v] for v in sorted(sim)])

NameError: name 'props' is not defined

In [ ]:
import networkx as nx
import numpy as np

# The graph to visualize
G = nx.cycle_graph(20)

# 3d spring layout
pos = nx.spring_layout(G, dim=3, seed=779)
# Extract node and edge positions from the layout
node_xyz = np.array([pos[v] for v in sorted(G)])
edge_xyz = np.array([(pos[u], pos[v]) for u, v in G.edges()])

# Create the 3D figure
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")

# Plot the nodes - alpha is scaled by "depth" automatically
ax.scatter(*node_xyz.T, s=100, ec="w")

# Plot the edges
for vizedge in edge_xyz:
    ax.plot(*vizedge.T, color="tab:gray")


def _format_axes(ax):
    """Visualization options for the 3D axes."""
    # Turn gridlines off
    ax.grid(False)
    # Suppress tick labels
    for dim in (ax.xaxis, ax.yaxis, ax.zaxis):
        dim.set_ticks([])
    # Set axes labels
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("z")


_format_axes(ax)
fig.tight_layout()
plt.show()
